In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from datetime import date, timedelta
import datetime as dt

In [300]:
# data taken from Opendata.dc.gov
crash = pd.read_csv('Data/Crashes_in_the_district_of_Columbia.csv')

In [301]:
#there were a ton of columns that were completely null.
crash.head()

,﻿X,Y,CRASHID,CRIMEID,ISREPORTONSCENE,WASMAJORCRASHNOTIFIED,MAJORCRASHNOTIFIEDDATE,MAJORCRASHNOTIFIEDPERSONID,SCHOOLBUSRELATED,ISJUNCTIONINTERCHANGEAREA,...,PEDESTRIANSINVOLVED,MINORINJURIES,MAJORINJURIES,FATALITIES,TRAFFICCONTROLDEVICES,ADDRESS_ID,STREETSEGID,OBJECTID,XCOORD,YCOORD
0,-76.931799,38.888110,284561699.0,284564121.0,1,null,NaN,null,null,null,...,NaN,NaN,NaN,NaN,NaN,16410.0,9948.0,1,405917.240000,135586.72000
1,NaN,NaN,284558848.0,284558858.0,0,null,NaN,null,null,null,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN
2,-76.973458,38.933115,284558851.0,284558872.0,1,0,NaN,null,null,null,...,NaN,NaN,1.0,NaN,NaN,904915.0,2228.0,3,402301.519392,140580.85136
3,-76.989447,38.927668,284558855.0,177355172.0,1,null,NaN,null,null,null,...,NaN,NaN,NaN,NaN,NaN,803716.0,3325.0,4,400915.220000,139975.86000
4,-77.044894,38.906458,284558864.0,284558871.0,1,0,NaN,null,null,null,...,NaN,NaN,NaN,NaN,NaN,812026.0,10320.0,5,396106.240000,137622.30000


##### Columns I am actually interested in

In [302]:
crashs= crash[['\xef\xbb\xbfX', 'Y', 'CRASHID', 'STREETLIGHTING', 'LIGHTCONDITION', 'WEATHER', 'REPORTDATE', 
               'CYCLISTSINVOLVED', 'PEDESTRIANSINVOLVED', 'MINORINJURIES', 'MAJORINJURIES', 'FATALITIES' ]]

##### Data Cleansing

In [303]:
# Filling nan values 
crashs['FATALITIES'].fillna(value = 0, inplace = True)
crashs['MAJORINJURIES'].fillna(value = 0, inplace = True)
crashs['MINORINJURIES'].fillna(value = 0, inplace = True)
crashs['PEDESTRIANSINVOLVED'].fillna(value = 0, inplace = True)
crashs['CYCLISTSINVOLVED'].fillna(value = 0, inplace = True)

In [304]:
# Renaming X and Y which are actually Longitude and Latitude
crashs.rename(columns = {'\xef\xbb\xbfX': 'LONGITUDE', 'Y':'LATITUDE'}, inplace = True)

In [305]:
# dropping observations where the report data was not recorded
crashs.dropna(subset=['REPORTDATE'], inplace = True)

/Users/samuelstack/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [309]:
# saving resulted dataframe
crashs.to_csv('Crashes_data.csv')

In [ ]:
# removing some scraggly data from the date object
date_list = crashs['REPORTDATE'].tolist()

new_date = []
for date in date_list:
    new = date.replace('T', ' ' )
    new = new.replace('Z', '')
    new_date.append(new)
crashs['DATE']=new_date

In [ ]:
crashs.drop('REPORTDATE', inplace = True, axis = 1)

In [308]:
crashs['EVENT'] = 'CRASH'

/Users/samuelstack/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


Creating Crash dataframe for a Granger Causaility Test.
GCT takes two timeseries variables or features and compares them with lag to see if one has an affect on the other.
I will run a GCT on every unique Locations so I will need both Crashes and Tickets as my features to compare, Data to represent my timeseries and location information so i can create unique locations.

In [220]:
gra_crash = crashs[['LONGITUDE','LATITUDE','DATE','EVENT']]
#gra_crash[['LONGITUDE','LATITUDE']]=gra_crash[['LONGITUDE','LATITUDE']].round(4)

In [223]:
gra_crash.dropna(inplace = True)

/Users/samuelstack/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [225]:
gra_crash.head()

,LONGITUDE,LATITUDE,DATE,EVENT
0,-76.9318,38.8881,2009-01-11 00:00:00.000,CRASH
2,-76.9735,38.9331,2009-01-05 00:00:00.000,CRASH
4,-77.0449,38.9065,2008-08-03 00:00:00.000,CRASH
5,-76.9366,38.8869,2008-08-15 00:00:00.000,CRASH
6,-77.0341,38.9274,2009-06-26 00:00:00.000,CRASH


A formal data dictionary describing what each of the values means was not provided.  
These are my best guesses.  

##### Insights:
- Of the 150,000 crashes, only 220 of them confirm that alcohol was a factor. 
- DC has its own coordinate system because Longitude and latitude were not good enough.
- This set of information contains an exceptionally high amount columns that are completely null and columns that are mostly null.  Its like they are trying to make their data look like more than it actually is.
- All my Moving violations data is somewhere between 1100mb and 1200mb.

In [3]:
# Initially I planned as possibly encorporating this in as a factor to predict areas with crashs.
em_work = pd.read_csv('Data/Emergency_Work_Requests_via_DDOT_TOPs.csv')
# I have not found a practical way to encorporate this yet. 

/Users/samuelstack/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# This was suppose to be reports by residents of DC of safety concerns/issues that could also have some end analysis implications.
vzs = pd.read_csv('Data/Vision_Zero_Safety_Transportation.csv')
# I probably wont we able to incorporate this for time reasons.  

All the ticket data came in monthly Increments and had to be merged by hand (aka Python)
Fair Warning it is a long list of the same exact thing for 7 years.

### 2009

In [3]:
# Import
jan2009 = pd.read_csv('Data/Moving Violation data/01 January/Moving_Violations_in_January_2009.csv')
feb2009 = pd.read_csv('Data/Moving Violation data/02 February/Moving_Violations_in_February_2009.csv')
mar2009 = pd.read_csv('Data/Moving Violation data/03 March/Moving_Violations_in_March_2009.csv')
apr2009 = pd.read_csv('Data/Moving Violation data/04 April/Moving_Violations_in_April_2009.csv')
may2009 = pd.read_csv('Data/Moving Violation data/05 May/Moving_Violations_in_May_2009.csv')
jun2009 = pd.read_csv('Data/Moving Violation data/06 June/Moving_Violations_in_June_2009.csv')
jul2009 = pd.read_csv('Data/Moving Violation data/07 July/Moving_Violations_in_July_2009.csv')
aug2009 = pd.read_csv('Data/Moving Violation data/08 August/Moving_Violations_in_August_2009.csv')
sep2009 = pd.read_csv('Data/Moving Violation data/09 September/Moving_Violations_in_September_2009.csv')
oct2009 = pd.read_csv('Data/Moving Violation data/10 October/Moving_Violations_in_October_2009.csv')
nov2009 = pd.read_csv('Data/Moving Violation data/11 November/Moving_Violations_in_November_2009.csv')
dec2009 = pd.read_csv('Data/Moving Violation data/12 December/Moving_Violations_in_December_2009.csv')

In [5]:
jan2009.columns.values

array(['\xef\xbb\xbfX', 'Y', 'OBJECTID', 'ROW_', 'LOCATION', 'ADDRESS_ID',
       'STREETSEGID', 'XCOORD', 'YCOORD', 'TICKETTYPE', 'FINEAMT',
       'TOTALPAID', 'PENALTY1', 'PENALTY2', 'ACCIDENTINDICATOR',
       'AGENCYID', 'TICKETISSUEDATE', 'VIOLATIONCODE', 'VIOLATIONDESC',
       'ROW_ID'], dtype=object)

In [7]:
dataframes2009 = [JAN2009, FEB2009, MAR2009, APR2009, MAY2009, JUN2009, JUL2009, AUG2009, SEP2009, OCT2009, NOV2009, DEC2009]

In [6]:
# Desired Fields (Granger Causaility Test)
JAN2009 = jan2009[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
FEB2009 = feb2009[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
MAR2009 = mar2009[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
APR2009 = apr2009[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
MAY2009 = may2009[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
JUN2009 = jun2009[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
JUL2009 = jul2009[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
AUG2009 = aug2009[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
SEP2009 = sep2009[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
OCT2009 = oct2009[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
NOV2009 = nov2009[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
DEC2009 = dec2009[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]



In [15]:
df2009_1 = JAN2009.append([FEB2009,MAR2009,APR2009])
df2009_2 = MAY2009.append([JUN2009,JUL2009,AUG2009])
df2009_3 = SEP2009.append([OCT2009,NOV2009,DEC2009])
df2009 = df2009_1.append([df2009_2, df2009_3])
df2009.to_csv('moving_violations2009.csv')

Doing the same cleaning for every frame and feature engineering would be a drag.  Created a sweet function to do things for me.

In [206]:
def is_trap_f(df):
    is_trap =[]
    ticket_type = df['TICKETTYPE'].tolist()
    for ticket in ticket_type:
        if ticket == 'Photo':
            is_trap.append(1)
        else:
            is_trap.append(0)
    df['TRAP'] =is_trap
        

In [247]:
for year in dataframes2009:
    is_trap_f(year)

/Users/samuelstack/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [14]:
for year in dataframes2009:
    year.rename(columns ={'\xef\xbb\xbfX': 'LONGITUDE', 'Y': 'LATITUDE'}, inplace = True)
    year[['LONGITUDE',"LATITUDE"]] = year[['LONGITUDE',"LATITUDE"]].round(3) 

### 2010
The 2010 December csv on opendata.dc.gov had some issues when i was concatting by year and I could not download it.

In [11]:
jan2010 = pd.read_csv('Data/Moving Violation data/01 January/Moving_Violations_in_January_2010.csv')
feb2010 = pd.read_csv('Data/Moving Violation data/02 February/Moving_Violations_in_February_2010.csv')
mar2010 = pd.read_csv('Data/Moving Violation data/03 March/Moving_Violations_in_March_2010.csv')
apr2010 = pd.read_csv('Data/Moving Violation data/04 April/Moving_Violations_in_April_2010.csv')
may2010 = pd.read_csv('Data/Moving Violation data/05 May/Moving_Violations_in_May_2010.csv')
jun2010 = pd.read_csv('Data/Moving Violation data/06 June/Moving_Violations_in_June_2010.csv')
jul2010 = pd.read_csv('Data/Moving Violation data/07 July/Moving_Violations_in_July_2010.csv')
aug2010 = pd.read_csv('Data/Moving Violation data/08 August/Moving_Violations_in_August_2010.csv')
sep2010 = pd.read_csv('Data/Moving Violation data/09 September/Moving_Violations_in_September_2010.csv')
oct2010 = pd.read_csv('Data/Moving Violation data/10 October/Moving_Violations_in_October_2010.csv')
nov2010 = pd.read_csv('Data/Moving Violation data/11 November/Moving_Violations_in_November_2010.csv')
#dec2010 = pd.read_csv('Data/Moving Violation data/12 December/Moving_Violations_in_December_2010.csv') This dataset is not downloadable

In [12]:
# Desired Fields (Granger Causaility Test)
JAN2010 = jan2010[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
FEB2010 = feb2010[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
MAR2010 = mar2010[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
APR2010 = apr2010[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
MAY2010 = may2010[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
JUN2010 = jun2010[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
JUL2010 = jul2010[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
AUG2010 = aug2010[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
SEP2010 = sep2010[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
OCT2010 = oct2010[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
NOV2010 = nov2010[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
#DEC2010 = dec2010[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE']]

In [13]:
dataframes2010 = [JAN2010, FEB2010, MAR2010, APR2010, MAY2010, JUN2010, JUL2010, AUG2010, SEP2010, OCT2010, NOV2010]

In [255]:
for year in dataframes2010:
    is_trap_f(year)

/Users/samuelstack/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [16]:
for year in dataframes2010:
    year.rename(columns ={'\xef\xbb\xbfX': 'LONGITUDE', 'Y': 'LATITUDE'}, inplace = True)
    year[['LONGITUDE',"LATITUDE"]] = year[['LONGITUDE',"LATITUDE"]].round(3) 

In [17]:
df2010_1 = JAN2010.append([FEB2010,MAR2010,APR2010])
df2010_2 = MAY2010.append([JUN2010,JUL2010,AUG2010])
df2010_3 = SEP2010.append([OCT2010,NOV2010])
df2010 = df2010_1.append([df2010_2, df2010_3])
df2010.to_csv('moving_violations2010.csv')

### 2011

In [18]:
jan2011 = pd.read_csv('Data/Moving Violation data/01 January/Moving_Violations_in_January_2011.csv')
feb2011 = pd.read_csv('Data/Moving Violation data/02 February/Moving_Violations_in_February_2011.csv')
mar2011 = pd.read_csv('Data/Moving Violation data/03 March/Moving_Violations_in_March_2011.csv')
apr2011 = pd.read_csv('Data/Moving Violation data/04 April/Moving_Violations_in_April_2011.csv')
may2011 = pd.read_csv('Data/Moving Violation data/05 May/Moving_Violations_in_May_2011.csv')
jun2011 = pd.read_csv('Data/Moving Violation data/06 June/Moving_Violations_in_June_2011.csv')
jul2011 = pd.read_csv('Data/Moving Violation data/07 July/Moving_Violations_in_July_2011.csv')
aug2011 = pd.read_csv('Data/Moving Violation data/08 August/Moving_Violations_in_August_2011.csv')
sep2011 = pd.read_csv('Data/Moving Violation data/09 September/Moving_Violations_in_September_2011.csv')
oct2011 = pd.read_csv('Data/Moving Violation data/10 October/Moving_Violations_in_October_2011.csv')
nov2011 = pd.read_csv('Data/Moving Violation data/11 November/Moving_Violations_in_November_2011.csv')
dec2011 = pd.read_csv('Data/Moving Violation data/12 December/Moving_Violations_in_December_2011.csv')

# Desired Fields (Granger Causaility Test)
JAN2011 = jan2011[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
FEB2011 = feb2011[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
MAR2011 = mar2011[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
APR2011 = apr2011[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
MAY2011 = may2011[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
JUN2011 = jun2011[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
JUL2011 = jul2011[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
AUG2011 = aug2011[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
SEP2011 = sep2011[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
OCT2011 = oct2011[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
NOV2011 = nov2011[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
DEC2011 = dec2011[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]

dataframes2011 = [JAN2011, FEB2011, MAR2011, APR2011, MAY2011, JUN2011, JUL2011, AUG2011, SEP2011, OCT2011, NOV2011, DEC2011]

# for year in dataframes2011:
#     is_trap_f(year)
    
for year in dataframes2011:
    year.rename(columns ={'\xef\xbb\xbfX': 'LONGITUDE', 'Y': 'LATITUDE'}, inplace = True)
    year[['LONGITUDE',"LATITUDE"]] = year[['LONGITUDE',"LATITUDE"]].round(3) 
    
df2011_1 = JAN2011.append([FEB2011,MAR2011,APR2011])
df2011_2 = MAY2011.append([JUN2011,JUL2011,AUG2011])
df2011_3 = SEP2011.append([OCT2011,NOV2011,DEC2011])
df2011 = df2011_1.append([df2011_2, df2011_3])
df2011.to_csv('moving_violations2011.csv')

In [19]:
df2011.head()

,LONGITUDE,LATITUDE,TICKETTYPE,TICKETISSUEDATE,FINEAMT,ACCIDENTINDICATOR,VIOLATIONDESC
0,-77.006,38.927,Photo,2011-01-28T00:00:00.000Z,125.0,No,SPEED 11-15 MPH OVER THE SPEED LIMIT
1,-76.995,38.879,Photo,2011-01-28T00:00:00.000Z,125.0,No,SPEED 11-15 MPH OVER THE SPEED LIMIT
2,-76.996,38.912,Photo,2011-01-28T00:00:00.000Z,125.0,No,SPEED 11-15 MPH OVER THE SPEED LIMIT
3,-76.965,38.897,Photo,2011-01-30T00:00:00.000Z,200.0,No,SPEED 21-25 MPH OVER THE SPEED LIMIT
4,-76.996,38.912,Photo,2011-01-30T00:00:00.000Z,125.0,No,SPEED 11-15 MPH OVER THE SPEED LIMIT


### 2012

In [20]:
jan2012 = pd.read_csv('Data/Moving Violation data/01 January/Moving_Violations_in_January_2012.csv')
feb2012 = pd.read_csv('Data/Moving Violation data/02 February/Moving_Violations_in_February_2012.csv')
mar2012 = pd.read_csv('Data/Moving Violation data/03 March/Moving_Violations_in_March_2012.csv')
apr2012 = pd.read_csv('Data/Moving Violation data/04 April/Moving_Violations_in_April_2012.csv')
may2012 = pd.read_csv('Data/Moving Violation data/05 May/Moving_Violations_in_May_2012.csv')
jun2012 = pd.read_csv('Data/Moving Violation data/06 June/Moving_Violations_in_June_2012.csv')
jul2012 = pd.read_csv('Data/Moving Violation data/07 July/Moving_Violations_in_July_2012.csv')
aug2012 = pd.read_csv('Data/Moving Violation data/08 August/Moving_Violations_in_August_2012.csv')
sep2012 = pd.read_csv('Data/Moving Violation data/09 September/Moving_Violations_in_September_2012.csv')
oct2012 = pd.read_csv('Data/Moving Violation data/10 October/Moving_Violations_in_October_2012.csv')
nov2012 = pd.read_csv('Data/Moving Violation data/11 November/Moving_Violations_in_November_2012.csv')
dec2012 = pd.read_csv('Data/Moving Violation data/12 December/Moving_Violations_in_December_2012.csv')

# Desired Fields (Granger Causaility Test)
JAN2012 = jan2012[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
FEB2012 = feb2012[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
MAR2012 = mar2012[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
APR2012 = apr2012[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
MAY2012 = may2012[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
JUN2012 = jun2012[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
JUL2012 = jul2012[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
AUG2012 = aug2012[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
SEP2012 = sep2012[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
OCT2012 = oct2012[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
NOV2012 = nov2012[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
DEC2012 = dec2012[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]

dataframes2012 = [JAN2012, FEB2012, MAR2012, APR2012, MAY2012, JUN2012, JUL2012, AUG2012, SEP2012, OCT2012, NOV2012, DEC2012]

# for year in dataframes2012:
#     is_trap_f(year)
    
for year in dataframes2012:
    year.rename(columns ={'\xef\xbb\xbfX': 'LONGITUDE', 'Y': 'LATITUDE'}, inplace = True)
    year[['LONGITUDE',"LATITUDE"]] = year[['LONGITUDE',"LATITUDE"]].round(3) 
    
df2012_1 = JAN2012.append([FEB2012,MAR2012,APR2012])
df2012_2 = MAY2012.append([JUN2012,JUL2012,AUG2012])
df2012_3 = SEP2012.append([OCT2012,NOV2012,DEC2012])
df2012 = df2012_1.append([df2012_2, df2012_3])
df2012.to_csv('moving_violations2012.csv')

### 2013

In [22]:
jan2013 = pd.read_csv('Data/Moving Violation data/01 January/Moving_Violations_in_January_2013.csv')
feb2013 = pd.read_csv('Data/Moving Violation data/02 February/Moving_Violations_in_February_2013.csv')
mar2013 = pd.read_csv('Data/Moving Violation data/03 March/Moving_Violations_in_March_2013.csv')
apr2013 = pd.read_csv('Data/Moving Violation data/04 April/Moving_Violations_in_April_2013.csv')
may2013 = pd.read_csv('Data/Moving Violation data/05 May/Moving_Violations_in_May_2013.csv')
jun2013 = pd.read_csv('Data/Moving Violation data/06 June/Moving_Violations_in_June_2013.csv')
jul2013 = pd.read_csv('Data/Moving Violation data/07 July/Moving_Violations_in_July_2013.csv')
aug2013 = pd.read_csv('Data/Moving Violation data/08 August/Moving_Violations_in_August_2013.csv')
sep2013 = pd.read_csv('Data/Moving Violation data/09 September/Moving_Violations_in_September_2013.csv')
oct2013 = pd.read_csv('Data/Moving Violation data/10 October/Moving_Violations_in_October_2013.csv')
nov2013 = pd.read_csv('Data/Moving Violation data/11 November/Moving_Violations_in_November_2013.csv')
dec2013 = pd.read_csv('Data/Moving Violation data/12 December/Moving_Violations_in_December_2013.csv')

# Desired Fields (Granger Causaility Test)
JAN2013 = jan2013[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
FEB2013 = feb2013[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
MAR2013 = mar2013[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
APR2013 = apr2013[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
MAY2013 = may2013[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
JUN2013 = jun2013[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
JUL2013 = jul2013[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
AUG2013 = aug2013[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
SEP2013 = sep2013[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
OCT2013 = oct2013[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
NOV2013 = nov2013[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
DEC2013 = dec2013[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]

dataframes2013 = [JAN2013, FEB2013, MAR2013, APR2013, MAY2013, JUN2013, JUL2013, AUG2013, SEP2013, OCT2013, NOV2013, DEC2013]

# for year in dataframes2013:
#     is_trap_f(year)
    
for year in dataframes2013:
    year.rename(columns ={'\xef\xbb\xbfX': 'LONGITUDE', 'Y': 'LATITUDE'}, inplace = True)
    year[['LONGITUDE',"LATITUDE"]] = year[['LONGITUDE',"LATITUDE"]].round(3) 
    
df2013_1 = JAN2013.append([FEB2013,MAR2013,APR2013])
df2013_2 = MAY2013.append([JUN2013,JUL2013,AUG2013])
df2013_3 = SEP2013.append([OCT2013,NOV2013,DEC2013])
df2013 = df2013_1.append([df2013_2, df2013_3])
df2013.to_csv('moving_violations2013.csv')

### 2014

In [23]:
jan2014 = pd.read_csv('Data/Moving Violation data/01 January/Moving_Violations_in_January_2014.csv')
feb2014 = pd.read_csv('Data/Moving Violation data/02 February/Moving_Violations_in_February_2014.csv')
mar2014 = pd.read_csv('Data/Moving Violation data/03 March/Moving_Violations_in_March_2014.csv')
apr2014 = pd.read_csv('Data/Moving Violation data/04 April/Moving_Violations_in_April_2014.csv')
may2014 = pd.read_csv('Data/Moving Violation data/05 May/Moving_Violations_in_May_2014.csv')
jun2014 = pd.read_csv('Data/Moving Violation data/06 June/Moving_Violations_in_June_2014.csv')
jul2014 = pd.read_csv('Data/Moving Violation data/07 July/Moving_Violations_in_July_2014.csv')
aug2014 = pd.read_csv('Data/Moving Violation data/08 August/Moving_Violations_in_August_2014.csv')
sep2014 = pd.read_csv('Data/Moving Violation data/09 September/Moving_Violations_in_September_2014.csv')
oct2014 = pd.read_csv('Data/Moving Violation data/10 October/Moving_Violations_in_October_2014.csv')
nov2014 = pd.read_csv('Data/Moving Violation data/11 November/Moving_Violations_in_November_2014.csv')
dec2014 = pd.read_csv('Data/Moving Violation data/12 December/Moving_Violations_in_December_2014.csv')

# Desired Fields (Granger Causaility Test)
JAN2014 = jan2014[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
FEB2014 = feb2014[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
MAR2014 = mar2014[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
APR2014 = apr2014[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
MAY2014 = may2014[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
JUN2014 = jun2014[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
JUL2014 = jul2014[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
AUG2014 = aug2014[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
SEP2014 = sep2014[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
OCT2014 = oct2014[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
NOV2014 = nov2014[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
DEC2014 = dec2014[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]

dataframes2014 = [JAN2014, FEB2014, MAR2014, APR2014, MAY2014, JUN2014, JUL2014, AUG2014, SEP2014, OCT2014, NOV2014, DEC2014]

# for year in dataframes2014:
#     is_trap_f(year)
    
for year in dataframes2014:
    year.rename(columns ={'\xef\xbb\xbfX': 'LONGITUDE', 'Y': 'LATITUDE'}, inplace = True)
    year[['LONGITUDE',"LATITUDE"]] = year[['LONGITUDE',"LATITUDE"]].round(3) 
    
df2014_1 = JAN2014.append([FEB2014,MAR2014,APR2014])
df2014_2 = MAY2014.append([JUN2014,JUL2014,AUG2014])
df2014_3 = SEP2014.append([OCT2014,NOV2014,DEC2014])
df2014 = df2014_1.append([df2014_2, df2014_3])
df2014.to_csv('moving_violations2014.csv')

### 2015

In [24]:
jan2015 = pd.read_csv('Data/Moving Violation data/01 January/Moving_Violations_in_January_2015.csv')
feb2015 = pd.read_csv('Data/Moving Violation data/02 February/Moving_Violations_in_February_2015.csv')
mar2015 = pd.read_csv('Data/Moving Violation data/03 March/Moving_Violations_in_March_2015.csv')
apr2015 = pd.read_csv('Data/Moving Violation data/04 April/Moving_Violations_in_April_2015.csv')
may2015 = pd.read_csv('Data/Moving Violation data/05 May/Moving_Violations_in_May_2015.csv')
jun2015 = pd.read_csv('Data/Moving Violation data/06 June/Moving_Violations_in_June_2015.csv')
jul2015 = pd.read_csv('Data/Moving Violation data/07 July/Moving_Violations_in_July_2015.csv')
aug2015 = pd.read_csv('Data/Moving Violation data/08 August/Moving_Violations_in_August_2015.csv')
sep2015 = pd.read_csv('Data/Moving Violation data/09 September/Moving_Violations_in_September_2015.csv')
oct2015 = pd.read_csv('Data/Moving Violation data/10 October/Moving_Violations_in_October_2015.csv')
nov2015 = pd.read_csv('Data/Moving Violation data/11 November/Moving_Violations_in_November_2015.csv')
dec2015 = pd.read_csv('Data/Moving Violation data/12 December/Moving_Violations_in_December_2015.csv')

# Desired Fields (Granger Causaility Test)
JAN2015 = jan2015[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
FEB2015 = feb2015[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
MAR2015 = mar2015[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
APR2015 = apr2015[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
MAY2015 = may2015[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
JUN2015 = jun2015[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
JUL2015 = jul2015[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
AUG2015 = aug2015[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
SEP2015 = sep2015[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
OCT2015 = oct2015[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
NOV2015 = nov2015[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
DEC2015 = dec2015[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]

dataframes2015 = [JAN2015, FEB2015, MAR2015, APR2015, MAY2015, JUN2015, JUL2015, AUG2015, SEP2015, OCT2015, NOV2015, DEC2015]

# for year in dataframes2015:
#     is_trap_f(year)
    
for year in dataframes2015:
    year.rename(columns ={'\xef\xbb\xbfX': 'LONGITUDE', 'Y': 'LATITUDE'}, inplace = True)
    year[['LONGITUDE',"LATITUDE"]] = year[['LONGITUDE',"LATITUDE"]].round(3) 
    
df2015_1 = JAN2015.append([FEB2015,MAR2015,APR2015])
df2015_2 = MAY2015.append([JUN2015,JUL2015,AUG2015])
df2015_3 = SEP2015.append([OCT2015,NOV2015,DEC2015])
df2015 = df2015_1.append([df2015_2, df2015_3])
df2015.to_csv('moving_violations2015.csv')

In [27]:
jan2016 = pd.read_csv('Data/Moving Violation data/01 January/Moving_Violations_in_January_2016.csv')
feb2016 = pd.read_csv('Data/Moving Violation data/02 February/Moving_Violations_in_February_2016.csv')
mar2016 = pd.read_csv('Data/Moving Violation data/03 March/Moving_Violations_in_March_2016.csv')
apr2016 = pd.read_csv('Data/Moving Violation data/04 April/Moving_Violations_in_April_2016.csv')
may2016 = pd.read_csv('Data/Moving Violation data/05 May/Moving_Violations_in_May_2016.csv')
# jun2016 = pd.read_csv('Data/Moving Violation data/06 June/Moving_Violations_in_June_2016.csv')
# jul2016 = pd.read_csv('Data/Moving Violation data/07 July/Moving_Violations_in_July_2016.csv')
# aug2016 = pd.read_csv('Data/Moving Violation data/08 August/Moving_Violations_in_August_2016.csv')
# sep2016 = pd.read_csv('Data/Moving Violation data/09 September/Moving_Violations_in_September_2016.csv')
# oct2016 = pd.read_csv('Data/Moving Violation data/10 October/Moving_Violations_in_October_2016.csv')
# nov2016 = pd.read_csv('Data/Moving Violation data/11 November/Moving_Violations_in_November_2016.csv')
# dec2016 = pd.read_csv('Data/Moving Violation data/12 December/Moving_Violations_in_December_2016.csv')

# Desired Fields (Granger Causaility Test)
JAN2016 = jan2016[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
FEB2016 = feb2016[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
MAR2016 = mar2016[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
APR2016 = apr2016[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
MAY2016 = may2016[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
# JUN2016 = jun2016[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
# JUL2016 = jul2016[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
# AUG2016 = aug2016[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
# SEP2016 = sep2016[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
# OCT2016 = oct2016[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
# NOV2016 = nov2016[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]
# DEC2016 = dec2016[['\xef\xbb\xbfX', 'Y','TICKETTYPE','TICKETISSUEDATE','FINEAMT','ACCIDENTINDICATOR','VIOLATIONDESC']]

dataframes2016 = [JAN2016, FEB2016, MAR2016, APR2016, MAY2016]

#, JUN2016, JUL2016, AUG2016, SEP2016, OCT2016, NOV2016, DEC2016
# for year in dataframes2016:
#     is_trap_f(year)
    
for year in dataframes2016:
    year.rename(columns ={'\xef\xbb\xbfX': 'LONGITUDE', 'Y': 'LATITUDE'}, inplace = True)
    year[['LONGITUDE',"LATITUDE"]] = year[['LONGITUDE',"LATITUDE"]].round(3) 
    
df2016_1 = JAN2016.append([FEB2016,MAR2016,APR2016,MAY2016])
# df2016_2 = MAY2016.append([JUN2016,JUL2016,AUG2016])
# df2016_3 = SEP2016.append([OCT2016,NOV2016,DEC2016])
#df2016 = df2016_1.append([df2016_2])
df2016_1.to_csv('moving_violations2016.csv')

#### Because of how longitude and latitude are calculate i decided it be more computationally efficient and accurate to round the Longitude and Latitude coordinates to broader positions.    This measurement is good for identifying intersections or areas on a road.

- The sign tells us whether we are north or south, east or west on the globe.
- A nonzero hundreds digit tells us we're using longitude, not latitude!
- The tens digit gives a position to about 1,000 kilometers. It gives us useful information about what continent or ocean we are on.
- The units digit (one decimal degree) gives a position up to 111 kilometers (60 nautical miles, about 69 miles). It can tell us roughly what large state or country we are in.
- The first decimal place is worth up to 11.1 km: it can distinguish the position of one large city from a neighboring large city.
- The second decimal place is worth up to 1.1 km: it can separate one village from the next.
- The third decimal place is worth up to 110 m: it can identify a large agricultural field or institutional campus.
##### - The fourth decimal place is worth up to 11 m: it can identify a parcel of land. It is comparable to the typical accuracy of an uncorrected GPS unit with no interference.
- The fifth decimal place is worth up to 1.1 m: it distinguish trees from each other. Accuracy to this level with commercial GPS units can only be achieved with differential correction.
- The sixth decimal place is worth up to 0.11 m: you can use this for laying out structures in detail, for designing landscapes, building roads. It should be more than good enough for tracking movements of glaciers and rivers. This can be achieved by taking painstaking measures with GPS, such as differentially corrected GPS.
- The seventh decimal place is worth up to 11 mm: this is good for much surveying and is near the limit of what GPS-based techniques can achieve.
- The eighth decimal place is worth up to 1.1 mm: this is good for charting motions of tectonic plates and movements of volcanoes. Permanent, corrected, constantly-running GPS base stations might be able to achieve this level of accuracy.
- The ninth decimal place is worth up to 110 microns: we are getting into the range of microscopy. For almost any conceivable application with earth positions, this is overkill and will be more precise than the accuracy of any surveying device.
- Ten or more decimal places indicates a computer or calculator was used and that no attention was paid to the fact that the extra decimals are useless. Be careful, because unless you are the one reading these numbers off the device, this can indicate low quality processing!

#### Insights
- T119 --  Speeding 11-15 mph over limit
- T120 --  Speeding 16-20 mph over limit
- T113 --  Red Light
- T121 --  Speeding 21-25 mph over limit

In [109]:
#I used this data for a month of Exploratory analysis in Tableau
tab_jan = JAN2009[['LONGITUDE','LATITUDE','DATE','EVENT']]
tab_crash = crashs[['LONGITUDE','LATITUDE','DATE','EVENT']]

In [112]:
tab_data= tab_jan.append(tab_crash)

In [118]:
date_L = tab_data['DATE']
dayt = []
for time in date_L:
    nt = time.replace('.000', '')
    dayt.append(nt)
    
tab_data['DATE1']=dayt

In [120]:
tab_data.drop('DATE', axis = 1, inplace =True)

In [129]:
tab_data.drop([6210,114802,7557,63007, 102937, 10669, 32350,76145], inplace = True)

In [164]:
#tab_data.sort(columns = 'DATE')
# Issues converting to datatime brought up that there are some invalid dates in these datasets.
# 2 crashes from 1908, another from 1975
# Also, 5 Crashes that happened in the future, 2017, 2025, 2028, 2320 and 2912
# Manually removed these rows, which is the series of index ids in the above cell.

In [134]:
# Converting to datetime 
t = tab_data['DATE1']
tdt = pd.to_datetime(t, format = "%Y-%m-%d %H:%M:%S")
tab_data['DATE'] = tdt
tab_data.drop('DATE1', inplace = True, axis =1)
# did this by creating a list, modifying list, appending new list as column and dropping the old one.

In [144]:
# As this data is for exploratoy analysis I will drop all null data.
tab_data.dropna(inplace = True)

In [163]:
# Don't have any ticket information prior to 2009 so removing all records that were previous.
tab_data = tab_data[(tab_data['DATE'].dt.year >= 2009)]